In [35]:
import pandas as pd
import numpy as np


train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

col_year = []
col_month = []
col_day = []

time_df = train_df['TIMESTAMP']

for i in range(len(time_df)):
    
    time_df_i = time_df[i]
    
    year = time_df_i[:4]
    col_year.append(year)

    month = time_df_i[5:7]
    col_month.append(month)

    day = time_df_i[8:10]
    col_day.append(day)

train_df['year'] = col_year
train_df['month'] = col_month
train_df['day'] = col_day

In [38]:
cor_time_y = train_df[['Y_Class','year','month','day']]